<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 9.7: Text Classification
INSTRUCTIONS:
- Run the cells
- Observe and understand the results
- Answer the questions

## Import libraries

In [42]:
## Import Libraries
import numpy as np
import pandas as pd
import re

import string
import spacy

from collections import Counter

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

# import warnings
# warnings.filterwarnings('ignore')

## Load data

Sample:

    __label__2 Stuning even for the non-gamer: This sound ...
    __label__2 The best soundtrack ever to anything.: I'm ...
    __label__2 Amazing!: This soundtrack is my favorite m ...
    __label__2 Excellent Soundtrack: I truly like this so ...
    __label__2 Remember, Pull Your Jaw Off The Floor Afte ...
    __label__2 an absolute masterpiece: I am quite sure a ...
    __label__1 Buyer beware: This is a self-published boo ...
    . . .
    
There are only two **labels**:
- `__label__1`
- `__label__2`

In [3]:
## Loading the data

trainDF = pd.read_fwf(
    filepath_or_buffer = 'corpus.txt',
    colspecs = [(9, 10),   # label: get only the numbers 1 or 2
                (11, 9000) # text: makes the it big enought to get to the end of the line
               ], 
    header = 0,
    names = ['label', 'text'],
    lineterminator = '\n'
)

# convert label from [1, 2] to [0, 1]
trainDF['label'] = trainDF['label'] - 1

## Inspect the data

In [4]:
# ANSWER
trainDF

,label,text
0,1,The best soundtrack ever to anything.: I'm rea...
1,1,Amazing!: This soundtrack is my favorite music...
2,1,Excellent Soundtrack: I truly like this soundt...
3,1,"Remember, Pull Your Jaw Off The Floor After He..."
4,1,an absolute masterpiece: I am quite sure any o...
...,...,...
9994,1,A revelation of life in small town America in ...
9995,1,Great biography of a very interesting journali...
9996,0,Interesting Subject; Poor Presentation: You'd ...
9997,0,Don't buy: The box looked used and it is obvio...


## Split the data into train and test

In [5]:
## ANSWER
## split the dataset
X_train,X_test,y_train,y_test = train_test_split(trainDF.text,trainDF.label, test_size = 0.2,random_state =0)

## Feature Engineering

### Count Vectors as features

In [6]:
# create a count vectorizer object
count_vect = CountVectorizer(token_pattern = r'\w{1,}')

# Learn a vocabulary dictionary of all tokens in the raw documents
count_vect.fit(trainDF['text'])

# Transform documents to document-term matrix.
X_train_count = count_vect.transform(X_train)
X_test_count = count_vect.transform(X_test)

### TF-IDF Vectors as features
- Word level
- N-Gram level
- Character level

In [7]:
%%time
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer = 'word',
                             token_pattern = r'\w{1,}',
                             max_features = 5000)
print(tfidf_vect)

tfidf_vect.fit(trainDF['text'])
X_train_tfidf = tfidf_vect.transform(X_train)
X_test_tfidf  = tfidf_vect.transform(X_test)

TfidfVectorizer(max_features=5000, token_pattern='\\w{1,}')
CPU times: user 860 ms, sys: 10.6 ms, total: 870 ms
Wall time: 876 ms


In [8]:
%%time
# ngram level tf-idf
tfidf_vect_ngram = TfidfVectorizer(analyzer = 'word',
                                   token_pattern = r'\w{1,}',
                                   ngram_range = (2, 3),
                                   max_features = 5000)
print(tfidf_vect_ngram)

tfidf_vect_ngram.fit(trainDF['text'])
X_train_tfidf_ngram = tfidf_vect_ngram.transform(X_train)
X_test_tfidf_ngram  = tfidf_vect_ngram.transform(X_test)

TfidfVectorizer(max_features=5000, ngram_range=(2, 3), token_pattern='\\w{1,}')
CPU times: user 3.69 s, sys: 145 ms, total: 3.84 s
Wall time: 3.86 s


In [9]:
%%time
# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer = 'char',
                                         token_pattern = r'\w{1,}',
                                         ngram_range = (2, 3),
                                         max_features = 5000)
print(tfidf_vect_ngram_chars)

tfidf_vect_ngram_chars.fit(trainDF['text'])
X_train_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(X_train)
X_test_tfidf_ngram_chars  = tfidf_vect_ngram_chars.transform(X_test)

TfidfVectorizer(analyzer='char', max_features=5000, ngram_range=(2, 3),
                token_pattern='\\w{1,}')


/Users/JohanNg/miniconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:506: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


CPU times: user 6.36 s, sys: 110 ms, total: 6.47 s
Wall time: 6.47 s


### Text / NLP based features

Create some other features.

Char_Count = Number of Characters in Text

Word Count = Number of Words in Text

Word Density = Average Number of Char in Words

Punctuation Count = Number of Punctuation in Text

Title Word Count = Number of Words in Title

Uppercase Word Count = Number of Upperwords in Text

In [10]:
import string

In [46]:
trainDF

,label,text,Char_Count,word_count,char_count,word_density,punctuation_count,uppercase_word_count
0,1,The best soundtrack ever to anything.: I'm rea...,509,97,509,5.247423,14,12
1,1,Amazing!: This soundtrack is my favorite music...,760,129,760,5.891473,40,27
2,1,Excellent Soundtrack: I truly like this soundt...,743,118,743,6.296610,33,59
3,1,"Remember, Pull Your Jaw Off The Floor After He...",481,87,481,5.528736,22,31
4,1,an absolute masterpiece: I am quite sure any o...,825,142,825,5.809859,35,21
...,...,...,...,...,...,...,...,...
9994,1,A revelation of life in small town America in ...,867,152,867,5.703947,25,18
9995,1,Great biography of a very interesting journali...,861,141,861,6.106383,14,16
9996,0,Interesting Subject; Poor Presentation: You'd ...,650,108,650,6.018519,17,12
9997,0,Don't buy: The box looked used and it is obvio...,135,27,135,5.000000,6,4


In [15]:
#Char_Couunt
trainDF['char_count'] = trainDF['text'].apply(len)
trainDF['char_count']

0       509
1       760
2       743
3       481
4       825
       ... 
9994    867
9995    861
9996    650
9997    135
9998    536
Name: char_count, Length: 9999, dtype: int64

In [16]:
trainDF['word_count'] = trainDF['text'].apply(lambda x:len(str(x).split(" ")))

In [17]:
trainDF['word_density'] = trainDF.char_count/trainDF.word_count

In [18]:
count = lambda l1,l2: sum([1 for x in l1 if x in l2])

trainDF['punctuation_count'] = trainDF['text'].apply(lambda x: count(x,string.punctuation))

In [19]:
trainDF['uppercase_word_count']= trainDF['text'].apply(lambda x: sum(1 for a in x if a.isupper()))

In [71]:
%%time
# ANSWER
trainDF['title_word_count'] = trainDF['text'].apply(lambda x: len(str(re.findall(r'^(.*?):',x)).split()))

CPU times: user 75.3 ms, sys: 7.38 ms, total: 82.7 ms
Wall time: 81.2 ms


In [56]:
test = "The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny."

In [68]:
len(str(re.findall(r'^(.*?):',test)).split())

6

In [72]:
trainDF

,label,text,Char_Count,word_count,char_count,word_density,punctuation_count,uppercase_word_count,title_word_count
0,1,The best soundtrack ever to anything.: I'm rea...,509,97,509,5.247423,14,12,6
1,1,Amazing!: This soundtrack is my favorite music...,760,129,760,5.891473,40,27,1
2,1,Excellent Soundtrack: I truly like this soundt...,743,118,743,6.296610,33,59,2
3,1,"Remember, Pull Your Jaw Off The Floor After He...",481,87,481,5.528736,22,31,10
4,1,an absolute masterpiece: I am quite sure any o...,825,142,825,5.809859,35,21,3
...,...,...,...,...,...,...,...,...,...
9994,1,A revelation of life in small town America in ...,867,152,867,5.703947,25,18,11
9995,1,Great biography of a very interesting journali...,861,141,861,6.106383,14,16,7
9996,0,Interesting Subject; Poor Presentation: You'd ...,650,108,650,6.018519,17,12,4
9997,0,Don't buy: The box looked used and it is obvio...,135,27,135,5.000000,6,4,2


In [22]:
## load spaCy
nlp = spacy.load('en_core_web_sm')

import collections

from collections import Counter

Part of Speech in **SpaCy**

    POS   DESCRIPTION               EXAMPLES
    ----- ------------------------- ---------------------------------------------
    ADJ   adjective                 big, old, green, incomprehensible, first
    ADP   adposition                in, to, during
    ADV   adverb                    very, tomorrow, down, where, there
    AUX   auxiliary                 is, has (done), will (do), should (do)
    CONJ  conjunction               and, or, but
    CCONJ coordinating conjunction  and, or, but
    DET   determiner                a, an, the
    INTJ  interjection              psst, ouch, bravo, hello
    NOUN  noun                      girl, cat, tree, air, beauty
    NUM   numeral                   1, 2017, one, seventy-seven, IV, MMXIV
    PART  particle                  's, not,
    PRON  pronoun                   I, you, he, she, myself, themselves, somebody
    PROPN proper noun               Mary, John, London, NATO, HBO
    PUNCT punctuation               ., (, ), ?
    SCONJ subordinating conjunction if, while, that
    SYM   symbol                    $, %, §, ©, +, −, ×, ÷, =, :), 😝
    VERB  verb                      run, runs, running, eat, ate, eating
    X     other                     sfpksdpsxmsa
    SPACE space
    
Find out number of Adjective, Adverb, Noun, Numeric, Pronoun, Proposition, Verb.

    Hint:
    1. Convert text to spacy document
    2. Use pos_
    3. Use Counter 

In [17]:
# Initialise some columns for feature's counts
trainDF['adj_count'] = 0
trainDF['adv_count'] = 0
trainDF['noun_count'] = 0
trainDF['num_count'] = 0
trainDF['pron_count'] = 0
trainDF['propn_count'] = 0
trainDF['verb_count'] = 0

In [77]:
%%time
# ANSWER
for i in range(trainDF.shape[0]):
    doc = nlp(trainDF.iloc[i]['text'])
    c = Counter([t.pos_ for t in doc])
    
    trainDF.at[i, 'adj_count'] = c['ADJ']
    trainDF.at[i, 'adv_count'] = c['ADV']
    trainDF.at[i, 'noun_count'] = c['NOUN']
    trainDF.at[i, 'num_count'] = c['NUM']
    trainDF.at[i, 'pron_count'] = c['PRON']
    trainDF.at[i, 'propn_count'] = c['PROPN']
    trainDF.at[i, 'verb_count'] = c['VERB']

CPU times: user 2min 11s, sys: 641 ms, total: 2min 12s
Wall time: 2min 14s


In [78]:
cols = [
    'char_count', 'word_count', 'word_density',
    'punctuation_count', 'title_word_count',
    'uppercase_word_count', 'adj_count',
    'adv_count', 'noun_count', 'num_count',
    'pron_count', 'propn_count', 'verb_count']

trainDF[cols].sample(5)

,char_count,word_count,word_density,punctuation_count,title_word_count,uppercase_word_count,adj_count,adv_count,noun_count,num_count,pron_count,propn_count,verb_count
2429,831,148,5.614865,28,1,33,8.0,13.0,27.0,0.0,16.0,13.0,16.0
3649,296,57,5.192982,15,1,12,4.0,5.0,12.0,2.0,8.0,2.0,10.0
6885,656,123,5.333333,19,7,12,14.0,11.0,23.0,3.0,13.0,3.0,15.0
5459,178,29,6.137931,9,3,4,7.0,2.0,7.0,1.0,2.0,0.0,3.0
8115,655,116,5.646552,23,4,24,15.0,5.0,22.0,1.0,7.0,12.0,12.0


### Topic Models as features

In [20]:
%%time
# train a LDA Model
lda_model = LatentDirichletAllocation(n_components = 20, learning_method = 'online', max_iter = 20)

X_topics = lda_model.fit_transform(X_train_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

CPU times: user 36.6 s, sys: 96.4 ms, total: 36.7 s
Wall time: 36.7 s


In [21]:
# view the topic models
n_top_words = 10
topic_summaries = []
print('Group Top Words')
print('-----', '-'*80)
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    top_words = ' '.join(topic_words)
    topic_summaries.append(top_words)
    print('  %3d %s' % (i, top_words))

Group Top Words
----- --------------------------------------------------------------------------------
    0 joyce wake haiku philosophy western harsh movements accept social represents
    1 strictly baseball burmese secure squares voodoo michigan external gordon concealer
    2 describes dickens link buddy pony essence status dummy hopkins underground
    3 ear attempt jawbone fit headset 13 blade health emma jabra
    4 400 monitor 23 fireman 64 burns santa teenagers wal analog
    5 c boots incredible economics smith lesson van hi happening titanic
    6 episode terrific reach distant patrick wilderness narrator shore stewart manon
    7 air bed ray blu design manson pump mattress built pillow
    8 ego organized fortunately patti nb neanderthal neanderthals antonio biblical platform
    9 the i it and to a this for is my
   10 dr process pratchett cgi commercial richard 18 requires jesus sg
   11 drip eddie carter bebel grainy violin unrated schwartz ca disappeared
   12 la de cha

## Modelling

In [22]:
## helper function

def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)

    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)

    return accuracy_score(predictions, y_test)

In [23]:
# Keep the results in a dataframe
results = pd.DataFrame(columns = ['Count Vectors',
                                  'WordLevel TF-IDF',
                                  'N-Gram Vectors',
                                  'CharLevel Vectors'])

### Naive Bayes Classifier

In [24]:
%%time
# Naive Bayes on Count Vectors
accuracy1 = train_model(MultinomialNB(), X_train_count, y_train, X_test_count)
print('NB, Count Vectors    : %.4f\n' % accuracy1)

NB, Count Vectors    : 0.8395

CPU times: user 16 ms, sys: 4.88 ms, total: 20.9 ms
Wall time: 19.3 ms


In [25]:
%%time
# Naive Bayes on Word Level TF IDF Vectors
accuracy2 = train_model(MultinomialNB(), X_train_tfidf, y_train, X_test_tfidf)
print('NB, WordLevel TF-IDF : %.4f\n' % accuracy2)

NB, WordLevel TF-IDF : 0.8435

CPU times: user 13.1 ms, sys: 6.69 ms, total: 19.8 ms
Wall time: 17.2 ms


In [26]:
%%time
# Naive Bayes on Ngram Level TF IDF Vectors
accuracy3 = train_model(MultinomialNB(), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('NB, N-Gram Vectors   : %.4f\n' % accuracy3)

NB, N-Gram Vectors   : 0.8375

CPU times: user 8.55 ms, sys: 1.65 ms, total: 10.2 ms
Wall time: 8.55 ms


In [27]:
%%time
# # Naive Bayes on Character Level TF IDF Vectors
accuracy4 = train_model(MultinomialNB(), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('NB, CharLevel Vectors: %.4f\n' % accuracy4)

NB, CharLevel Vectors: 0.8130

CPU times: user 29 ms, sys: 28.8 ms, total: 57.8 ms
Wall time: 55.7 ms


In [28]:
results.loc['Naïve Bayes'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

### Linear Classifier

In [29]:
%%time
# Linear Classifier on Count Vectors
accuracy1 = train_model(LogisticRegression(solver = 'lbfgs', max_iter = 350), X_train_count, y_train, X_test_count)
print('LR, Count Vectors    : %.4f\n' % accuracy1)

LR, Count Vectors    : 0.8625

CPU times: user 1.84 s, sys: 4.24 s, total: 6.08 s
Wall time: 1.16 s


In [30]:
%%time
# Linear Classifier on Word Level TF IDF Vectors
accuracy2 = train_model(LogisticRegression(solver = 'lbfgs', max_iter = 100), X_train_tfidf, y_train, X_test_tfidf)
print('LR, WordLevel TF-IDF : %.4f\n' % accuracy2)

LR, WordLevel TF-IDF : 0.8610

CPU times: user 121 ms, sys: 2.16 ms, total: 123 ms
Wall time: 122 ms


In [31]:
%%time
# Linear Classifier on Ngram Level TF IDF Vectors
accuracy3 = train_model(LogisticRegression(solver = 'lbfgs', max_iter = 100), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('LR, N-Gram Vectors   : %.4f\n' % accuracy3)

LR, N-Gram Vectors   : 0.8455

CPU times: user 67.1 ms, sys: 1.92 ms, total: 69 ms
Wall time: 67.4 ms


In [32]:
%%time
# Linear Classifier on Character Level TF IDF Vectors
accuracy4 = train_model(LogisticRegression(solver = 'lbfgs', max_iter = 100), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('LR, CharLevel Vectors: %.4f\n' % accuracy4)

LR, CharLevel Vectors: 0.8430

CPU times: user 317 ms, sys: 22.5 ms, total: 339 ms
Wall time: 336 ms


In [33]:
results.loc['Logistic Regression'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

### Support Vector Machine

In [34]:
%%time
# Support Vector Machine on Count Vectors
accuracy1 = train_model(LinearSVC(), X_train_count, y_train, X_test_count)
print('SVM, Count Vectors    : %.4f\n' % accuracy1)

SVM, Count Vectors    : 0.8365

CPU times: user 433 ms, sys: 9.54 ms, total: 443 ms
Wall time: 442 ms


/Users/JohanNg/miniconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [35]:
%%time
# Support Vector Machine on Word Level TF IDF Vectors
accuracy2 = train_model(LinearSVC(), X_train_tfidf, y_train, X_test_tfidf)
print('SVM, WordLevel TF-IDF : %.4f\n' % accuracy2)

SVM, WordLevel TF-IDF : 0.8670

CPU times: user 70.1 ms, sys: 2.32 ms, total: 72.4 ms
Wall time: 69.5 ms


In [36]:
%%time
# Support Vector Machine on Ngram Level TF IDF Vectors
accuracy3 = train_model(LinearSVC(), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('SVM, N-Gram Vectors   : %.4f\n' % accuracy3)

SVM, N-Gram Vectors   : 0.8370

CPU times: user 49.9 ms, sys: 1.56 ms, total: 51.5 ms
Wall time: 50.5 ms


In [37]:
%%time
# Support Vector Machine on Character Level TF IDF Vectors
accuracy4 = train_model(LinearSVC(), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('SVM, CharLevel Vectors: %.4f\n' % accuracy4)

SVM, CharLevel Vectors: 0.8310

CPU times: user 282 ms, sys: 25.3 ms, total: 307 ms
Wall time: 306 ms


In [38]:
results.loc['Support Vector Machine'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

### Bagging Models

In [39]:
%%time
# Bagging (Random Forest) on Count Vectors
accuracy1 = train_model(RandomForestClassifier(n_estimators = 100), X_train_count, y_train, X_test_count)
print('RF, Count Vectors    : %.4f\n' % accuracy1)

RF, Count Vectors    : 0.8425

CPU times: user 8.22 s, sys: 19.2 ms, total: 8.24 s
Wall time: 8.24 s


In [40]:
%%time
# Bagging (Random Forest) on Word Level TF IDF Vectors
accuracy2 = train_model(RandomForestClassifier(n_estimators = 100), X_train_tfidf, y_train, X_test_tfidf)
print('RF, WordLevel TF-IDF : %.4f\n' % accuracy2)

RF, WordLevel TF-IDF : 0.8295

CPU times: user 4.1 s, sys: 12.5 ms, total: 4.11 s
Wall time: 4.11 s


In [41]:
%%time
# Bagging (Random Forest) on Ngram Level TF IDF Vectors
accuracy3 = train_model(RandomForestClassifier(n_estimators = 100), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('RF, N-Gram Vectors   : %.4f\n' % accuracy3)

RF, N-Gram Vectors   : 0.7980

CPU times: user 4.17 s, sys: 11.7 ms, total: 4.18 s
Wall time: 4.18 s


In [42]:
%%time
# Bagging (Random Forest) on Character Level TF IDF Vectors
accuracy4 = train_model(RandomForestClassifier(n_estimators = 100), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('RF, CharLevel Vectors: %.4f\n' % accuracy4)

RF, CharLevel Vectors: 0.7945

CPU times: user 14.6 s, sys: 44.7 ms, total: 14.7 s
Wall time: 14.6 s


In [43]:
results.loc['Random Forest'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

### Boosting Models

In [44]:
%%time
# Gradient Boosting on Count Vectors
accuracy1 = train_model(GradientBoostingClassifier(), X_train_count, y_train, X_test_count)
print('GB, Count Vectors    : %.4f\n' % accuracy1)

GB, Count Vectors    : 0.8025

CPU times: user 12.9 s, sys: 27.2 ms, total: 13 s
Wall time: 13 s


In [45]:
%%time
# Gradient Boosting on Word Level TF IDF Vectors
accuracy2 = train_model(GradientBoostingClassifier(), X_train_tfidf, y_train, X_test_tfidf)
print('GB, WordLevel TF-IDF : %.4f\n' % accuracy2)

GB, WordLevel TF-IDF : 0.8000

CPU times: user 9.06 s, sys: 20.7 ms, total: 9.08 s
Wall time: 9.08 s


In [46]:
%%time
# Gradient Boosting on Ngram Level TF IDF Vectors
accuracy3 = train_model(GradientBoostingClassifier(), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('GB, N-Gram Vectors   : %.4f\n' % accuracy3)

GB, N-Gram Vectors   : 0.7525

CPU times: user 5.27 s, sys: 11.8 ms, total: 5.28 s
Wall time: 5.28 s


In [47]:
%%time
# Gradient Boosting on Character Level TF IDF Vectors
accuracy4 = train_model(GradientBoostingClassifier(), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('GB, CharLevel Vectors: %.4f\n' % accuracy4)

GB, CharLevel Vectors: 0.8155

CPU times: user 1min 25s, sys: 220 ms, total: 1min 26s
Wall time: 1min 26s


In [48]:
results.loc['Gradient Boosting'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

In [49]:
results

,Count Vectors,WordLevel TF-IDF,N-Gram Vectors,CharLevel Vectors
Naïve Bayes,0.8395,0.8435,0.8375,0.8130
Logistic Regression,0.8625,0.8610,0.8455,0.8430
Support Vector Machine,0.8365,0.8670,0.8370,0.8310
Random Forest,0.8425,0.8295,0.7980,0.7945
Gradient Boosting,0.8025,0.8000,0.7525,0.8155




---



---



> > > > > > > > > © 2021 Institute of Data


---



---



